In [1]:
#!/usr/bin/env python
# coding: utf-8

#------------------------------------------------------------------------------
#Set Seed
#------------------------------------------------------------------------------

import random

random.seed(2020)
print(random.random())

#------------------------------------------------------------------------------
#Detect number of cores on computer (needed for multicore processing to speed up code)
#------------------------------------------------------------------------------

import psutil
psutil.cpu_count()
psutil.cpu_count(logical=False)  # Ignoring virtual cores

#------------------------------------------------------------------------------
#Set working directory
#------------------------------------------------------------------------------
import os

#------------------------------------------------------------------------------
#Import relevant packages
#------------------------------------------------------------------------------

import pandas as pd
import os
from pprint import pprint
import csv

from dateutil.parser import parse
from pandas import DataFrame

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.ldamulticore import LdaMulticore

# spacy for lemmatization
import spacy
from nltk.stem.wordnet import WordNetLemmatizer

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline





0.6196692706606616


In [2]:
#------------------------------------------------------------------------------
#Prepare stopwords
#------------------------------------------------------------------------------

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
#stop_words.extend(['using', 'publication', 'ltd', 'elsevier', 'reserved', 'rights'])

#------------------------------------------------------------------------------
#Import newspaper articles
#------------------------------------------------------------------------------
import glob
import docx

#define blank list
words = []

#define function to import USAT, NYT, WaPo
def list_files(dir):                                                                                                  
    r = []                                                                                                            
    subdirs = [x[0] for x in os.walk(dir)]                                                                            
    for subdir in subdirs:                                                                                            
        files = os.walk(subdir).__next__()[2]                                                                             
        if (len(files) > 0):                                                                                          
            for file in files:
                if "Bibliography" in file:
                    continue
                if("doclist" in file):
                    continue
                if(".docx" in file):
                    r.append(os.path.join(subdir, file))                                                                         
    return r                                                                                                          


file_list = list_files("/Users/Mihir/Downloads/Sample/")
#file_list = list_files("/Users/anavirshermon/Dropbox (Kenan-Flagler)/Newspaper Analysis/Sample/")

#import wsj articles, which are in an excel sheet
wsj      = pd.read_excel("/Users/Mihir/Downloads/Sample/wsj.xlsx", engine='openpyxl') 
wsj      = wsj[wsj['Content'].notna()]

wsj['year'] = wsj['date'].dt.year #extract year

wsj_1    = wsj[(wsj['year']<=2012)]
wsj_2    = wsj[(wsj['year']>2012) & (wsj['year']<= 2015)]
wsj_3    = wsj[(wsj['year']>=2016)]

wsj_list   = wsj['Content'].tolist()
wsj_1_list = wsj_1['Content'].tolist()
wsj_2_list = wsj_2['Content'].tolist()
wsj_3_list = wsj_3['Content'].tolist()

    
#add words to files list
for i in file_list:
    doc = docx.Document(i)
    words.append([p.text for p in doc.paragraphs]) 
        
#making each word in paragraph formation and removing all words that come before "Body", which tells us when the paragraph begins
for i in range(len(words)):
    words[i] = ' '.join(words[i])
    #words[i] = words[i][words[0].index("Body "):]
    
one = []
two = [] 
three = []

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.9/site-packages/docx/section.py:7: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Sequence


In [3]:

#split up files between dates
for i in words:
    date =parse(i[i.index("Load-Date:") + 11:i.index("End of Document")])
    if(date.year<=2012):
        one.append(i)
    elif(date.year<=2015):
        two.append(i)
    else:
        three.append(i)

words_cleaned = [x.rsplit("Body ")[1] for x in words]
words_cleaned = [x.split("Load-Date")[0] for x in words_cleaned]
words_cleaned = [x for x in words_cleaned if str(x) != 'nan']

one_cleaned = [x.rsplit("Body ")[1] for x in one]
one_cleaned = [x.split("Load-Date")[0] for x in one_cleaned]
one_cleaned = [x for x in one_cleaned if str(x) != 'nan']

two_cleaned = [x.rsplit("Body ")[1] for x in two]
two_cleaned = [x.split("Load-Date")[0] for x in two_cleaned]
two_cleaned = [x for x in two_cleaned if str(x) != 'nan']

three_cleaned = [x.rsplit("Body ")[1] for x in three]
three_cleaned = [x.split("Load-Date")[0] for x in three_cleaned]
three_cleaned = [x for x in three_cleaned if str(x) != 'nan']

for i in wsj_list:
    words_cleaned.append(i)  

for i in wsj_1_list:
    one_cleaned.append(i)
    
for i in wsj_2_list:
    two_cleaned.append(i)
    
for i in wsj_3_list:
    three_cleaned.append(i)

#------------------------------------------------------------------------------
#Drop military related newspaper articles
#------------------------------------------------------------------------------

#Code to eventually use tf-idf or some systematic method to drop military articles
from pandas import DataFrame
import re

#drop articles if they contain these words
# =============================================================================
# drop_words    = {'military', 'strike', 'attack', 'syria', 'iran', 'pakistan'}
# words_cleaned = [i for i in words_cleaned if not any(x in i for x in drop_words)]
# one_cleaned   = [i for i in one_cleaned if not any(x in i for x in drop_words)]
# two_cleaned   = [i for i in two_cleaned if not any(x in i for x in drop_words)]
# two_cleaned = [i for i in three_cleaned if not any(x in i for x in drop_words)]
# 
# =============================================================================

drop_words= {'military', 'strike', 'attack', 'syria', 'yemen', 'pakistan', 'afghanistan', 'taliban', 'pentagon', 'islamic'}

#convert to lower case
data        = [item.lower() for item in words_cleaned]
data_one    = [item.lower() for item in one_cleaned]
data_two    = [item.lower() for item in two_cleaned]
data_three  = [item.lower() for item in three_cleaned]

#remove new line characters
#data = [re.sub('\s+', ' ', sent) for sent in data]   # Remove new line characters

#tokenize and pre-process the text
def split(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
data        = list(split(data))
data_one    = list(split(data_one))
data_two    = list(split(data_two))
data_three  = list(split(data_three))


#remove numbers, but not words that contain numbers
data       = [[token for token in doc if not token.isnumeric()] for doc in data]
data_one   = [[token for token in doc if not token.isnumeric()] for doc in data_one]
data_two   = [[token for token in doc if not token.isnumeric()] for doc in data_two]
data_three = [[token for token in doc if not token.isnumeric()] for doc in data_three]

#remove words that are only two characters
data       = [[token for token in doc if len(token) > 2] for doc in data]
data_one   = [[token for token in doc if len(token) > 2] for doc in data_one]
data_two   = [[token for token in doc if len(token) > 2] for doc in data_two]
data_three = [[token for token in doc if len(token) > 2] for doc in data_three]

#remove stopwords
def remove_stopwords(words):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in words]
data       = remove_stopwords(data)
data_one   = remove_stopwords(data_one)
data_two   = remove_stopwords(data_two)
data_three = remove_stopwords(data_three)


# =============================================================================
# from collections import Counter 
# test = Counter(c for clist in data for c in clist)
# most_occur = test.most_common(150)
# 
# =============================================================================


/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
print(len(data))
print(len(data_one))
print(len(data_two))
print(len(data_three))

2498
668
1121
709


/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
import nltk
from nltk.corpus import wordnet as wn
nouns = {x.name().split('.', 1)[0] for x in wn.all_synsets('n')}
verbs = {x.name().split('.', 1)[0] for x in wn.all_synsets('v')}


/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
for i in range(len(data)):
    data[i] = [w for w in data[i] if w in nouns]
for i in range(len(data_one)):
    data_one[i] = [w for w in data_one[i] if (w in nouns or w in verbs)]
for i in range(len(data_two)):
    data_two[i] = [w for w in data_two[i] if (w in nouns or w in verbs)]
for i in range(len(data_three)):
    data_three[i] = [w for w in data_three[i] if (w in nouns or w in verbs)]

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
'''
These lines combine all of the data into paragraph form so that it can be read by the tfidf vector class. They are then split back afterwards.
'''
data = [' '.join(w) for w in data]
data_one = [' '.join(w) for w in data_one]
data_two = [' '.join(w) for w in data_two]
data_three = [' '.join(w) for w in data_three]
loop = [data, data_one, data_two, data_three]

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
names = ["data", "data_one", "data_two", "data_three"]
group = [data, data_one, data_two, data_three]
d = {}
'''
Creates the td-idf vectors for the total (data), 2012-2015 (data_one), 2015-2017 (data_two), 2017+(data_three)
and stores in the a dictionary (d)
'''
for i, j in zip(names, group):
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform(j)
    feature_names = vectorizer.get_feature_names()
    dense = vectors.todense()
    denselist = dense.tolist()
    df = pd.DataFrame(denselist, columns=feature_names)
    d[i] = df

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
''' 
Splits data again 
'''
data = [i.split() for i in data]
data_one = [i.split() for i in data_one]
data_two = [i.split() for i in data_two]
data_three = [i.split() for i in data_three]
res = {}

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
#Gets the top 40 words for each of the tf-idf vectors and stores it in the dictionary (res)
for n, i in zip(names, d):    
    array = []
    for j in d[i].iterrows():
         array.append((j[1].sort_values(ascending=False)[:40].keys().tolist()))
    res[n] = array

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
#Drops document if the top 40 words is in drop_words array
data = [data[i] for i in range(len(data)) if not any (x in res['data_one'][i] for x in drop_words)]
data_one = [data_one[i] for i in range(len(data_one)) if not any (x in res['data_one'][i] for x in drop_words)]
data_two = [data_two[i] for i in range(len(data_two)) if not any (x in res['data_two'][i] for x in drop_words)]
data_three = [data_three[i] for i in range(len(data_three)) if not any (x in res['data_three'][i] for x in drop_words)]

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
#Saves data
pd.DataFrame(res['data']).to_csv("total.csv")

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
#!/usr/bin/env python3
# coding: utf-8

## Author: Eduardo Coronado (Duke University)

import gensim.corpora as corpora
from gensim.models import CoherenceModel
import sys

def train_HDPmodel(hdp, word_list, mcmc_iter, burn_in=100, quiet=False):
    '''Wrapper function to train tomotopy HDP Model object
    
    *** Inputs**
    hdp: obj -> initialized HDPModel model
    word_list: list -> lemmatized word list of lists
    mcmc_iter : int -> number of iterations to train the model
    burn_in: int -> MC burn in iterations
    quiet: bool -> flag whether to print iteration LL and Topics, if True nothing prints out
    
    ** Returns**
    hdp: trained HDP Model 
    '''
    
    # Add docs to train
    for vec in word_list:
        hdp.add_doc(vec)

    # Initiate MCMC burn-in 
    hdp.burn_in = 100
    hdp.train(0)
    print('Num docs:', len(hdp.docs), ', Vocab size:', hdp.num_vocabs, ', Num words:', hdp.num_words)
    print('Removed top words:', hdp.removed_top_words)
    print('Training...', file=sys.stderr, flush=True)

    # Train model
    step=round(mcmc_iter*0.10)
    for i in range(0, mcmc_iter, step):
        hdp.train(step, workers=3)
        if not quiet:
            print('Iteration: {}\tLog-likelihood: {}\tNum. of topics: {}'.format(i, hdp.ll_per_word, hdp.live_k))
        
    print("Done\n")  
    
    return hdp
    
        
def get_hdp_topics(hdp, top_n=10):
    '''Wrapper function to extract topics from trained tomotopy HDP model 
    
    ** Inputs **
    hdp:obj -> HDPModel trained model
    top_n: int -> top n words in topic based on frequencies
    
    ** Returns **
    topics: dict -> per topic, an arrays with top words and associated frequencies 
    '''
    
    # Get most important topics by # of times they were assigned (i.e. counts)
    sorted_topics = [k for k, v in sorted(enumerate(hdp.get_count_by_topics()), key=lambda x:x[1], reverse=True)]

    topics=dict()
    
    # For topics found, extract only those that are still assigned
    for k in sorted_topics:
        if not hdp.is_live_topic(k): continue # remove un-assigned topics at the end (i.e. not alive)
        topic_wp =[]
        for word, prob in hdp.get_topic_words(k, top_n=top_n):
            topic_wp.append((word, prob))

        topics[k] = topic_wp # store topic word/frequency array
        
    return topics



def eval_coherence(topics_dict, word_list, coherence_type='c_v'):
    '''Wrapper function that uses gensim Coherence Model to compute topic coherence scores
    
    ** Inputs **
    topic_dict: dict -> topic dictionary from train_HDPmodel function
    word_list: list -> lemmatized word list of lists
    coherence_typ: str -> type of coherence value to comput (see gensim for opts)
    
    ** Returns **
    score: float -> coherence value
    '''
    
    # Build gensim objects
    vocab = corpora.Dictionary(word_list)
    corpus = [vocab.doc2bow(words) for words in word_list]
    
    # Build topic list from dictionary
    topic_list=[]
    for k, tups in topics_dict.items():
        topic_tokens=[]
        for w, p in tups:
            topic_tokens.append(w)
            
        topic_list.append(topic_tokens)
            

    # Build Coherence model
    print("Evaluating topic coherence...")
    cm = CoherenceModel(topics=topic_list, corpus=corpus, dictionary=vocab, texts=word_list, 
                    coherence=coherence_type)
    
    score = cm.get_coherence()
    print ("Done\n")
    return score

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [26]:

#create bigrams and trigrams
bigram         = gensim.models.Phrases(data, min_count=5, threshold=10)
trigram        = gensim.models.Phrases(bigram[data], threshold=10)
bigram_mod     = gensim.models.phrases.Phraser(bigram)
trigram_mod    = gensim.models.phrases.Phraser(trigram)

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

data_bigrams    = make_bigrams(data)
data_trigrams   = make_trigrams(data)

data_one_trigrams   = make_trigrams(data_one)
data_two_trigrams   = make_trigrams(data_two)
data_three_trigrams = make_trigrams(data_three)

#lemmatize words
lemmatizer = WordNetLemmatizer()
data_lemmatized = [[lemmatizer.lemmatize(token) for token in doc] for doc in data_trigrams]

print(data_lemmatized[99:100])

data_one_lemmatized   = [[lemmatizer.lemmatize(token) for token in doc] for doc in data_one_trigrams]
data_two_lemmatized   = [[lemmatizer.lemmatize(token) for token in doc] for doc in data_two_trigrams]
data_three_lemmatized = [[lemmatizer.lemmatize(token) for token in doc] for doc in data_three_trigrams]

data_lemmatized = data_one_lemmatized

group = "_2013-2015"

id2word = gensim.corpora.Dictionary(data_lemmatized)

id2word.filter_extremes(no_below=20, no_above=0.75)

texts = data_lemmatized


/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[['administration', 'accounting', 'noncombatant', 'statistic', 'may', 'much', 'left', 'release_information', 'first', 'three', 'month', 'part', 'president', 'use_force', 'battle', 'terrorism', 'operation', 'speech_national_defense', 'university', 'subject', 'transparency_oversight', 'death', 'like', 'cover', 'war', 'yemen_somalia_libya', 'pakistan', 'drone', 'administration', 'part', 'afghanistan', 'war', 'theater', 'still', 'inside_pakistan', 'pentagon', 'saturday', 'taliban_leader', 'pakistan', 'must', 'level', 'government', 'special_operations', 'three', 'self_defense', 'defense_secretary', 'case', 'march', 'somalia', 'aircraft', 'training_camp', 'war', 'strike', 'may', 'african', 'peacekeeping', 'west', 'mogadishu', 'somali', 'capital', 'keep', 'administration', 'civilian', 'drone', 'statistic', 'threat', 'growth', 'state', 'standing', 'president', 'term', 'january', 'hope', 'time', 'leave', 'office', 'structure', 'place', 'set', 'process', 'people', 'look', 'government', 'basis', 

In [68]:
data_lemmatized = data_three_lemmatized
texts = data_lemmatized



/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [69]:
import tomotopy as tp
term_weight = tp.TermWeight.ONE
hdp = tp.HDPModel(tw=term_weight, min_cf=5, rm_top=7, gamma=1,
                  alpha=0.1, initial_k=10, seed=99999)

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [70]:
for vec in texts:
    hdp.add_doc(vec)
hdp.burn_in = 100
hdp.train(0)
print('Num docs:', len(hdp.docs), ', Vocab size:', hdp.num_vocabs,
      ', Num words:', hdp.num_words)
print('Removed top words:', hdp.removed_top_words)
for i in range(0, 1000, 100):
    hdp.train(100) # 100 iterations at a time
    print('Iteration: {}\tLog-likelihood: {}\tNum. of topics: {}'.format(i, hdp.ll_per_word, hdp.live_k))

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Num docs: 446 , Vocab size: 2800 , Num words: 68011
Removed top words: ['drone', 'one', 'company', 'technology', 'use', 'aircraft', 'people']
Iteration: 0	Log-likelihood: -7.630016002346397	Num. of topics: 31
Iteration: 100	Log-likelihood: -7.573190233157086	Num. of topics: 35
Iteration: 200	Log-likelihood: -7.568108922212357	Num. of topics: 34
Iteration: 300	Log-likelihood: -7.557782638576195	Num. of topics: 38
Iteration: 400	Log-likelihood: -7.558150741676118	Num. of topics: 38
Iteration: 500	Log-likelihood: -7.550722510197929	Num. of topics: 41
Iteration: 600	Log-likelihood: -7.548508014237391	Num. of topics: 38
Iteration: 700	Log-likelihood: -7.538801896139348	Num. of topics: 43
Iteration: 800	Log-likelihood: -7.531804622622746	Num. of topics: 45
Iteration: 900	Log-likelihood: -7.531854914156365	Num. of topics: 47


In [71]:

topics = get_hdp_topics(hdp, top_n=30) # changing top_n changes no. of words displayed


/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [72]:
res = {}
id = 0
for i in topics.values():
    res[id] = []
    for j in i:
        res[id].append(j[0])
    id+=1

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [73]:
pd.DataFrame(res).to_csv("post-2015.csv")

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
